# Actor-critic algorithm



The challenge is to experiment with the actor-critic algorithm.

## Requirements:
### 1. __TensorFlow:__
https://www.tensorflow.org/get_started/os_setup. GPU is not needed.
### 2. __OpenAI Gym:__  
OpenAI Gym - https://gym.openai.com/docs. Please, use version 0.10.5.
### 3. __TensorFlow-probabilities:__<br>
https://www.tensorflow.org/probability/install. 
***

Let's import everything we need and write auxiliary functions:

In [ ]:
%matplotlib inline

import numpy as np
import tensorflow as tf
print(tf.__version__)

import tensorflow_probability as tfp
import gym
print(gym.__version__)

import logz
import os
import time
import inspect
from multiprocessing import Process

import matplotlib.pyplot as plt
from IPython.display import clear_output

def pathlength(path):
    return len(path["reward"])

def setup_logger(logdir, locals_):
    # Configure output directory for logging
    logz.configure_output_dir(logdir)
    # Log experimental parameters
    args = inspect.getargspec(train_PG)[0]
    params = {k: locals_[k] if k in locals_ else None for k in args}
    logz.save_params(params)
    
x=[]
y=[]
fig=None
def plot(t, mean_r):
    global x
    global y 
    if t==0:
        x = []
        y = []
        fig=plt.figure()
    x.append(t)
    y.append(mean_r)
    clear_output(wait=True)
    plt.plot(x,y,label='mean_reward')
    plt.xlabel("Number of iterations")
    plt.ylabel("Reward")
    plt.legend()
    plt.show()


***
## Recall the necessary formulas

The gradient of the objective function is calcualted to determine the gradient of the strategy using the following formula:

$$\nabla_\theta J(\theta)\approx\frac{1}{N}\sum_{i=1}^{N}\sum_{j=1}^{T} \nabla_\theta\log\pi_\theta(a_{it},s_{it}) A^\pi(s_t,a_t).$$

Here we evaluate the utility function of the action through the advantage function $ A $, which, to reduce the variance of the solution, we present in the form of the difference between the critic's estimates and the base function $B(s)=V_v^\pi(s)$:

$$A^\pi(s_t,a_t)\approx r(a_t,s_t)+\gamma V_v^\pi(s_{t+1})-V_v^\pi(s_{it})$$

Critic who evaluates reward function minimizes standard error for TD metric $y_t=r(a_t,s_t)+\gamma V^\pi(s_{t+1})$:

$$\min_v \sum_{i,t}(V_v^\pi(s_{it})-y_{it})^2.$$

In order to speed up the work, we repeat the following two steps:

1. Update the metric with the old value of the utility function.
2. We take several gradient steps to update the utility function.

Those. The critic’s training is iterative: we update the criterion and then update the function itself.

### Create an approximator.

We need to create a simple approximator in the form of several layers (__n_layers__) of fully connected neural networks (__tf.layers.dense__). 

In [ ]:
def build_mlp(input_placeholder, output_size, scope, n_layers, size, activation=tf.tanh, output_activation=None):
    """
        Builds a feedforward neural network
        
        arguments:
            input_placeholder: placeholder variable for the state (batch_size, input_size)
            output_size: size of the output layer
            scope: variable scope of the network
            n_layers: number of hidden layers
            size: dimension of the hidden layer
            activation: activation of the hidden layers
            output_activation: activation of the ouput layers
        returns:
            output placeholder of the network (the result of a forward pass) 
        Hint: use tf.layers.dense    
    """
    with tf.variable_scope(scope + "1"):
        output_placeholder = input_placeholder
        for _ in range(n_layers):
            output_placeholder = tf.layers.dense(output_placeholder, size, activation=activation)
        output_placeholder = tf.layers.dense(output_placeholder, output_size, activation=output_activation)
    return output_placeholder

Next, we implement the core Agent class. First we define its constructor, the parameters of which are taken from the dictionary __computation_graph_args__. Parameters __computation_graph_args__ are auxiliary.

In [ ]:
class Agent(object):
    def __init__(self, computation_graph_args, sample_trajectory_args, estimate_advantage_args, scope):
        super(Agent, self).__init__()
        self.ob_dim = computation_graph_args['ob_dim']
        self.ac_dim = computation_graph_args['ac_dim']
        self.discrete = computation_graph_args['discrete']
        self.size = computation_graph_args['size']
        self.n_layers = computation_graph_args['n_layers']
        self.learning_rate = computation_graph_args['learning_rate']
        self.num_target_updates = computation_graph_args['num_target_updates']
        self.num_grad_steps_per_target_update = computation_graph_args['num_grad_steps_per_target_update']

        self.animate = sample_trajectory_args['animate']
        self.max_path_length = sample_trajectory_args['max_path_length']
        self.min_timesteps_per_batch = sample_trajectory_args['min_timesteps_per_batch']

        self.gamma = estimate_advantage_args['gamma']
        self.normalize_advantages = estimate_advantage_args['normalize_advantages']
        self.scope = scope
        
    def init_tf_sess(self):
        tf_config = tf.ConfigProto(inter_op_parallelism_threads=1, intra_op_parallelism_threads=1)
        tf_config.gpu_options.allow_growth = True # may need if using GPU
        self.sess = tf.Session(config=tf_config)
        self.sess.__enter__() # equivalent to `with self.sess:`
        tf.global_variables_initializer().run() #pylint: disable=E1101

    def define_placeholders(self):
        """
            Placeholders for batch batch observations / actions / advantages in actor critic
            loss function.
            See Agent.build_computation_graph for notation

            returns:
                sy_ob_no: placeholder for observations
                sy_ac_na: placeholder for actions
                sy_adv_n: placeholder for advantages
        """
        sy_ob_no = tf.placeholder(shape=[None, self.ob_dim], name="ob", dtype=tf.float32)
        if self.discrete:
            sy_ac_na = tf.placeholder(shape=[None], name="ac", dtype=tf.int32) 
        else:
            sy_ac_na = tf.placeholder(shape=[None, self.ac_dim], name="ac", dtype=tf.float32) 
        
        sy_adv_n = tf.placeholder(shape=[None],name="advantage",dtype=tf.float32)
        return sy_ob_no, sy_ac_na, sy_adv_n

***
### Implementation of the algorithm of the actor.

First, we need to add a code to determine the action for the current observation (__sy_ob_no__). This operation varies for continuous and discrete environments.

In [ ]:
def policy_forward_pass(self, sy_ob_no):
    """ Constructs the symbolic operation for the policy network outputs,
        which are the parameters of the policy distribution p(a|s)
        arguments:
            sy_ob_no: (batch_size, self.ob_dim)
        returns:
            the parameters of the policy.
            if discrete, the parameters are the logits of a categorical distribution
                over the actions
                sy_logits_na: (batch_size, self.ac_dim)
            if continuous, the parameters are a tuple (mean, log_std) of a Gaussian
                distribution over actions. log_std should just be a trainable
                variable, not a network output.
                sy_mean: (batch_size, self.ac_dim)
                sy_logstd: (self.ac_dim,)
        Hint: use the 'build_mlp' function to output the logits (in the discrete case)
            and the mean (in the continuous case).
            Pass in self.n_layers for the 'n_layers' argument, and
            pass in self.size for the 'size' argument.
    """
    
    if self.discrete:
        sy_logits_na = build_mlp(sy_ob_no,self.ac_dim,self.scope+"0",self.n_layers,self.size, activation=tf.nn.relu)
        return sy_logits_na
    else:
        sy_mean = build_mlp(sy_ob_no,self.ac_dim,self.scope+"0",self.n_layers,self.size,activation=tf.nn.relu)
        sy_logstd = tf.Variable(np.zeros(self.ac_dim),name="sy_logstd")
        return (sy_mean, sy_logstd)
    
Agent.policy_forward_pass = policy_forward_pass

Next, we select the needed action from the distribution, which is given to us by the strategy. For a discrete and continuous case, we use different methods (__tf.squeeze__ and __tf.random_normal__).

In [ ]:
def sample_action(self, policy_parameters):
    """ Constructs a symbolic operation for stochastically sampling from the policy
        distribution
        arguments:
            policy_parameters
                if discrete: logits of a categorical distribution over actions 
                    sy_logits_na: (batch_size, self.ac_dim)
                if continuous: (mean, log_std) of a Gaussian distribution over actions
                    sy_mean: (batch_size, self.ac_dim)
                    sy_logstd: (self.ac_dim,)
        returns:
            sy_sampled_ac: 
                if discrete: (batch_size,)
                if continuous: (batch_size, self.ac_dim)
        Hint: for the continuous case, use the reparameterization trick:
             The output from a Gaussian distribution with mean 'mu' and std 'sigma' is

                  mu + sigma * z,         z ~ N(0, I)

             This reduces the problem to just sampling z. (Hint: use tf.random_normal!)
    """
    if self.discrete:
        sy_logits_na = policy_parameters
        # use tf.squeeze (Removes dimensions of size 1 from the shape of a tensor.)
        sy_sampled_ac = tf.squeeze(tf.multinomial(sy_logits_na, 1), axis=1)
    else:
        sy_mean, sy_logstd = policy_parameters
        exp=tf.cast(tf.exp(sy_logstd), dtype=tf.float32)
        sy_sampled_ac = sy_mean + exp * tf.random_normal(shape=tf.shape(sy_mean))
    return sy_sampled_ac

Agent.sample_action = sample_action

Next, we calculate the logarithm of the probability of the action for use in the objective function.

In [ ]:
def get_log_prob(self, policy_parameters, sy_ac_na):
    """ Constructs a symbolic operation for computing the log probability of a set of actions
        that were actually taken according to the policy
        arguments:
            policy_parameters
                if discrete: logits of a categorical distribution over actions 
                    sy_logits_na: (batch_size, self.ac_dim)
                if continuous: (mean, log_std) of a Gaussian distribution over actions
                    sy_mean: (batch_size, self.ac_dim)
                    sy_logstd: (self.ac_dim,)
            sy_ac_na: 
                if discrete: (batch_size,)
                if continuous: (batch_size, self.ac_dim)
        returns:
            sy_logprob_n: (batch_size)
        Hint:
            For the discrete case, use the log probability under a categorical distribution.
            For the continuous case, use the log probability under a multivariate gaussian.
    """
    if self.discrete:
        sy_logits_na = policy_parameters
        # use tf.nn.sparse_softmax_cross_entropy_with_logits
        sy_logprob_n = - tf.nn.sparse_softmax_cross_entropy_with_logits(labels=sy_ac_na,logits=sy_logits_na)
    else:
        sy_mean, sy_logstd = policy_parameters
        # multivariate gaussian (tf.distributions.Normal)
        exp=tf.cast(tf.exp(sy_logstd),dtype=tf.float32)
        probabilities = tf.distributions.Normal(sy_mean, exp).prob(sy_ac_na)
        sy_logprob_n = tf.log(tf.reduce_prod(probabilities, axis=1))
    return sy_logprob_n

Agent.get_log_prob = get_log_prob

Finally, we write the code for updating the parameters of the actor in the session. Here __actor_update_op__ is the optimizer, which we define later.

In [ ]:
def update_actor(self, ob_no, ac_na, adv_n):
    """ 
        Update the parameters of the policy.

        arguments:
            ob_no: shape: (sum_of_path_lengths, ob_dim)
            ac_na: shape: (sum_of_path_lengths).
            adv_n: shape: (sum_of_path_lengths). A single vector for the estimated
                advantages whose length is the sum of the lengths of the paths

        returns:
            nothing

    """
    self.sess.run(self.actor_update_op,feed_dict={self.sy_ob_no: ob_no, self.sy_ac_na: ac_na, self.sy_adv_n: adv_n})
    
Agent.update_actor = update_actor

***
### Code for the critic.

We start by evaluating the advantage function (see the formulas at the beginning of the notebook).

In [ ]:
def estimate_advantage(self, ob_no, next_ob_no, re_n, terminal_n):
    """
        Estimates the advantage function value for each timestep.

        let sum_of_path_lengths be the sum of the lengths of the paths sampled from 
            Agent.sample_trajectories

        arguments:
            ob_no: shape: (sum_of_path_lengths, ob_dim)
            next_ob_no: shape: (sum_of_path_lengths, ob_dim). The observation after taking one step forward
            re_n: length: sum_of_path_lengths. Each element in re_n is a scalar containing
                the reward for each timestep
            terminal_n: length: sum_of_path_lengths. Each element in terminal_n is either 1 if the episode ended
                at that timestep of 0 if the episode did not end

        returns:
            adv_n: shape: (sum_of_path_lengths). A single vector for the estimated 
                advantages whose length is the sum of the lengths of the paths
    """
    # First, estimate the Q value as Q(s, a) = r(s, a) + gamma*V(s')
    # To get the advantage, subtract the V(s) to get A(s, a) = Q(s, a) - V(s)
    # This requires calling the critic twice --- to obtain V(s') when calculating Q(s, a),
    # and V(s) when subtracting the baseline
    # Note: don't forget to use terminal_n to cut off the V(s') term when computing Q(s, a)
    # otherwise the values will grow without bound.

    adv_n = []
    # calculate the value of the utility of the next state s' with the use of critic
    v_s_tp1_n = self.sess.run(self.critic_prediction, feed_dict={self.sy_ob_no: next_ob_no})
    # calculate the value of the utility of the current state s with the use of critic
    v_s_t_n = self.sess.run(self.critic_prediction, feed_dict={self.sy_ob_no: ob_no})
    # calculate the value Q (s, a) = r (s, a) + gamma * V (s')
    q_n = re_n + self.gamma * v_s_tp1_n * (1 - terminal_n)
    # A(s, a) = Q(s, a) - V(s)
    adv_n = q_n - v_s_t_n

    eps = 0.000000001 # not to devide by 0

    if self.normalize_advantages:
        # calculate the average and divide by the variance
        adv_n = (adv_n - np.mean(adv_n)) / (np.std(adv_n) + eps)
    return adv_n

Agent.estimate_advantage = estimate_advantage

Now it is necessary to write how we update the parameters of the critic in the session.

In [ ]:
def update_critic(self, ob_no, next_ob_no, re_n, terminal_n):
    """
        Update the parameters of the critic.

        let sum_of_path_lengths be the sum of the lengths of the paths sampled from
            Agent.sample_trajectories
        let num_paths be the number of paths sampled from Agent.sample_trajectories

        arguments:
            ob_no: shape: (sum_of_path_lengths, ob_dim)
            next_ob_no: shape: (sum_of_path_lengths, ob_dim). The observation after taking one step forward
            re_n: length: sum_of_path_lengths. Each element in re_n is a scalar containing
                the reward for each timestep
            terminal_n: length: sum_of_path_lengths. Each element in terminal_n is either 1 if the episode ended
                at that timestep of 0 if the episode did not end

        returns:
            nothing
    """
    # Use a bootstrapped target values to update the critic
    # Compute the target values r(s, a) + gamma*V(s') by calling the critic to compute V(s')
    # In total, take n=self.num_grad_steps_per_target_update*self.num_target_updates gradient update steps
    # Every self.num_grad_steps_per_target_update steps, recompute the target values
    # by evaluating V(s') on the updated critic
    # Note: don't forget to use terminal_n to cut off the V(s') term when computing the target
    # otherwise the values will grow without bound.

    for i in range(self.num_target_updates):
        target_n = []
        # calculate the value of the utility of the next state s' with the use of critic
        v_s_tp1_n = self.sess.run(self.critic_prediction, feed_dict={self.sy_ob_no: next_ob_no})
        # consider the value of the time difference index r (s, a) + gamma * V (s') if this is not a terminal state
        target_n = re_n + self.gamma * v_s_tp1_n * (1 - terminal_n)

        for j in range(self.num_grad_steps_per_target_update):
            self.sess.run(self.critic_update_op, feed_dict={self.sy_target_n: target_n,self.sy_ob_no: ob_no})

Agent.update_critic = update_critic

***
### Combination of methods

Now let's combine all the previous methods into workflow.

In [ ]:
def build_computation_graph(self):
    """
        Notes on notation:

        Symbolic variables have the prefix sy_, to distinguish them from the numerical values
        that are computed later in the function

        Prefixes and suffixes:
        ob - observation 
        ac - action
        _no - this tensor should have shape (batch self.size /n/, observation dim)
        _na - this tensor should have shape (batch self.size /n/, action dim)
        _n  - this tensor should have shape (batch self.size /n/)

        Note: batch self.size /n/ is defined at runtime, and until then, the shape for that axis
        is None

        ----------------------------------------------------------------------------------
        loss: a function of self.sy_logprob_n and self.sy_adv_n that we will differentiate
            to get the policy gradient.
    """
    self.sy_ob_no, self.sy_ac_na, self.sy_adv_n = self.define_placeholders()

    # The policy takes in an observation and produces a distribution over the action space
    self.policy_parameters = self.policy_forward_pass(self.sy_ob_no)

    # We can sample actions from this action distribution.
    # This will be called in Agent.sample_trajectory() where we generate a rollout.
    self.sy_sampled_ac = self.sample_action(self.policy_parameters)

    # We can also compute the logprob of the actions that were actually taken by the policy
    # This is used in the loss function.
    self.sy_logprob_n = self.get_log_prob(self.policy_parameters, self.sy_ac_na)

    actor_loss = tf.reduce_sum(-self.sy_logprob_n * self.sy_adv_n)
    self.actor_update_op = tf.train.AdamOptimizer(self.learning_rate).minimize(actor_loss)

    # define the critic
    self.critic_prediction = tf.squeeze(build_mlp(
                            self.sy_ob_no,
                            1,
                            self.scope + "1",
                            n_layers=self.n_layers,
                            size=self.size))
    self.sy_target_n = tf.placeholder(shape=[None], name="critic_target", dtype=tf.float32)
    self.critic_loss = tf.losses.mean_squared_error(self.sy_target_n, self.critic_prediction)
    self.critic_update_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.critic_loss)

Agent.build_computation_graph = build_computation_graph

Add  a couple more methods  to the agent of working with statistics.

In [ ]:
def sample_trajectories(self, itr, env):
    # Collect paths until we have enough timesteps
    timesteps_this_batch = 0
    paths = []
    while True:
        animate_this_episode=(len(paths)==0 and (itr % 10 == 0) and self.animate)
        path = self.sample_trajectory(env, animate_this_episode)
        paths.append(path)
        timesteps_this_batch += pathlength(path)
        if timesteps_this_batch > self.min_timesteps_per_batch:
            break
    return paths, timesteps_this_batch

def sample_trajectory(self, env, animate_this_episode):
    ob = env.reset()
    obs, acs, rewards, next_obs, terminals = [], [], [], [], []
    steps = 0
    while True:
        if animate_this_episode:
            env.render()
            time.sleep(0.1)
        obs.append(ob)
        ac = self.sess.run(self.sy_sampled_ac, feed_dict={self.sy_ob_no: ob[None, :]}) 
        ac = ac[0]
        acs.append(ac)
        ob, rew, done, _ = env.step(ac)
        # add the observation after taking a step to next_obs
        next_obs.append(ob)
        rewards.append(rew)
        steps += 1
        # If the episode ended, the corresponding terminal value is 1
        # otherwise, it is 0
        if done or steps > self.max_path_length:
            terminals.append(1)
            break
        else:
            terminals.append(0)
    path = {"observation" : np.array(obs, dtype=np.float32), 
            "reward" : np.array(rewards, dtype=np.float32), 
            "action" : np.array(acs, dtype=np.float32),
            "next_observation": np.array(next_obs, dtype=np.float32),
            "terminal": np.array(terminals, dtype=np.float32)}
    return path

Agent.sample_trajectories = sample_trajectories
Agent.sample_trajectory = sample_trajectory

***
### Let's carry out the experiments.

Below is the code in which we conduct experiments with the built agent. First, we initialize the environment (Set Up Env), then create an agent (Initialize Agent) and run it in the environment in a loop (Training Loop).

In [ ]:
def train_AC(
        exp_name,
        env_name,
        n_iter=100, 
        gamma=1, 
        min_timesteps_per_batch=1000, 
        max_path_length=None,
        learning_rate=5e-3,
        num_target_updates=10,
        num_grad_steps_per_target_update=10,
        animate=False, 
        normalize_advantages=True,
        seed=1,
        n_layers=2,
        size=64,
        scope="scope"):

    start = time.time()


    #========================================================================================#
    # Set Up Env
    #========================================================================================#

    # Make the gym environment
    env = gym.make(env_name)

    # Set random seeds
    tf.set_random_seed(seed)
    np.random.seed(seed)
    env.seed(seed)

    # Maximum length for episodes
    max_path_length = max_path_length or env.spec.max_episode_steps

    # Is this env continuous, or self.discrete?
    discrete = isinstance(env.action_space, gym.spaces.Discrete)

    # Observation and action sizes
    ob_dim = env.observation_space.shape[0]
    ac_dim = env.action_space.n if discrete else env.action_space.shape[0]

    #========================================================================================#
    # Initialize Agent
    #========================================================================================#

    computation_graph_args = {
        'n_layers': n_layers,
        'ob_dim': ob_dim,
        'ac_dim': ac_dim,
        'discrete': discrete,
        'size': size,
        'learning_rate': learning_rate,
        'num_target_updates': num_target_updates,
        'num_grad_steps_per_target_update': num_grad_steps_per_target_update,
        }

    sample_trajectory_args = {
        'animate': animate,
        'max_path_length': max_path_length,
        'min_timesteps_per_batch': min_timesteps_per_batch,
        }

    estimate_advantage_args = {
        'gamma': gamma,
        'normalize_advantages': normalize_advantages,
        }
    
    agent = Agent(computation_graph_args, sample_trajectory_args, estimate_advantage_args, scope=scope) #estimate_return_args

    # build computation graph
    agent.build_computation_graph()

    # tensorflow: config, session, variable initialization
    agent.init_tf_sess()

    #========================================================================================#
    # Training Loop
    #========================================================================================#

    total_timesteps = 0
    results = [[], []]
    for itr in range(n_iter):
        print("********** Iteration %i ************"%itr)
        paths, timesteps_this_batch = agent.sample_trajectories(itr, env)
        total_timesteps += timesteps_this_batch

        # Build arrays for observation, action for the policy gradient update by concatenating 
        # across paths
        ob_no = np.concatenate([path["observation"] for path in paths])
        ac_na = np.concatenate([path["action"] for path in paths])
        re_n = np.concatenate([path["reward"] for path in paths])
        next_ob_no = np.concatenate([path["next_observation"] for path in paths])
        terminal_n = np.concatenate([path["terminal"] for path in paths])

        # Call tensorflow operations to:
        # (1) update the critic, by calling agent.update_critic
        # (2) use the updated critic to compute the advantage by, calling agent.estimate_advantage
        # (3) use the estimated advantage values to update the actor, by calling agent.update_actor
        agent.update_critic(ob_no, next_ob_no, re_n, terminal_n)
        adv_n = agent.estimate_advantage(ob_no, next_ob_no, re_n, terminal_n)
        agent.update_actor(ob_no, ac_na, adv_n)
        
        # Log diagnostics
        returns = [path["reward"].sum() for path in paths]
        ep_lengths = [pathlength(path) for path in paths]
        print("Time", time.time() - start)
        print("Iteration", itr)
        print("AverageReturn", np.mean(returns))
        print("StdReturn", np.std(returns))
        print("MaxReturn", np.max(returns))
        print("MinReturn", np.min(returns))
        print("EpLenMean", np.mean(ep_lengths))
        print("EpLenStd", np.std(ep_lengths))
        print("TimestepsThisBatch", timesteps_this_batch)
        print("TimestepsSoFar", total_timesteps)
        plot(itr, np.mean(returns))
        results[0].append(itr)
        results[1].append(np.mean(returns))

    return results

Let's experiment with several environments and different parameters: __CartPole-v0__ (num_target_updates = [1,10,100], num_grad_steps_per_target_update = [1,10,100], max_path_length = 3000).


Let's choose the best combination for num_target_updates and num_grad_steps_per_target_update and check with these parameters __InvertedPendulum-v2__ (gamma=0.9, learning_rate=0.01, min_timesteps_per_batch=5000, size=64, max_path_length=3000), __HalfCheetah-v2__ (gamma=0.95, learning_rate=0.02, min_timesteps_per_batch=30000, size=32, max_path_length=3000). 

For each environment, we describe the features of the environment, construct graphs of the agent's quality of work, and comment on the results.

## Cartpole-v0

In [ ]:
cartpole_results = []

for num_target_updates in [1, 10, 100]:
    for num_grad_steps_per_target_update in [1, 10, 100]:
        res = train_AC(
                exp_name="CartPole-v0",
                env_name="CartPole-v0",
                num_target_updates=num_target_updates,
                num_grad_steps_per_target_update=num_grad_steps_per_target_update,
                scope='scope' + 'CartPole_v0' + str(num_target_updates) + str(num_grad_steps_per_target_update)
        )
        cartpole_results.append(res)

In [ ]:
plt.figure(figsize=(17, 20))

for i_col in range(3):
    for j_col in range(3):
        plt.subplots_adjust(top=0.9)
        plt.subplot(3, 3, 3*i_col+j_col+1)
        x_col = cartpole_results[i_col+j_col][0]
        y_col = cartpole_results[i_col+j_col][1]
        plt.plot(x_col, y_col)
        plt.xlabel('iteration')
        plt.ylabel('reward')
        plt.title('num_target_updates: {}\nnum_grad_steps_per_target_update: {}'.format(10**i_col, 10**j_col))

plt.show()

## Results

These graphs show that the maximum reward that we can potentially get in this environment is 200 cu Let us analyze some of them individually:

**1.1.** Really not quite good, even in 100 eras the result is unsatisfactory.

**1.10.** и **10.1** Here, the results are already much better – at nearly 25 iterations we are approaching the desired reward, but the graph is fluctuating

**10.10.** **1.100.** **100.1.** In this case, the reward begins to approach a reward of 200 a little later (at about 40 iterations), while the graph is almost steady.

**100.10.** **10.100.** Everything is fine, but at 80 iterations, reward goes down sharply and it is not clear whether it will come back or not.

**100.100.** Everything is great here: at about 15 iterations, the reward reaches the desired value and the graph generally does not flactuate - the best result.

## InvertedPendulum-v2

In [ ]:
# As we can see the best values of parameters num_target_updates and num_grad_steps_per_target_update are 100 and 100. 
# But it's too long and 10 and 10 give us almost the same reward so that
num_target_updates = 10
num_grad_steps_per_target_update = 10


In [ ]:
res_2 = train_AC(
                exp_name="InvertedPendulum-v2",
                env_name="InvertedPendulum-v2",
                n_iter=100,
                gamma=0.9,
                min_timesteps_per_batch=5000,
                max_path_length=3000,
                learning_rate=0.01,
                num_target_updates=num_target_updates,
                num_grad_steps_per_target_update=num_grad_steps_per_target_update,
                animate=False,
                normalize_advantages=False,
                seed=1,
                n_layers=2,
                size=64,
                scope='sc_____' + 'InvertedPendulum-v2'
        )

## Results

A very unstable chart, the values of the rewards are very different at different iterations. It can be concluded that models in this environment need to be trained for a very long time before convergence.

## HalfCheetah-v2 

In [ ]:
res_3 = train_AC(
                exp_name="HalfCheetah-v2",
                env_name="HalfCheetah-v2",
                n_iter=100,
                gamma=0.95,
                min_timesteps_per_batch=5000,
                max_path_length=3000,
                learning_rate=0.02,
                num_target_updates=num_target_updates,
                num_grad_steps_per_target_update=num_grad_steps_per_target_update,
                animate=False,
                normalize_advantages=False,
                seed=1,
                n_layers=2,
                size=32,
                scope='sc______' + 'InvertedPendulum-v2'
        )

## Results

In this environment, everything works quite well: the reward grows very quickly, the values of the reward do not fluctuate from iteration to iteration, quick convergence.